In [1]:
import pandas as pd
import sqlite3
import numpy as np
import requests
import re
from datetime import datetime
import time

import sys
import os

sys.path.append(os.getenv("ADMIN_PATH"))
import library

C:\Users\kodri\anaconda3\envs\citynet\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Create a league and teams selection
Retrieve links from xg tables and extract league and team info

In [ ]:
test

In [2]:
# Create a base from retrieved links to filter before committing
a = time.time()
links = library.retrievedLinks(library.bronze_path)
processor = library.processLinks(links)
processor.run()
last_retreived_date = str(links.date.max())
links = processor.df[['league_cleaned', 'teams_cleaned']].drop_duplicates()
print(f"{round(time.time() - a)} seconds to load")
league_selection = links.league_cleaned.unique()
team_selection = links.teams_cleaned.unique()
del links

34 seconds to load


### 2. Generate dates

In [3]:
# Create a list of missing links with a lag to allow for updates
dates = library.generate_dates_between(last_retreived_date, lag = 3)

### 3.  Retrieve links

In [4]:
processor = library.getLinks(dates)
processor.run()
data = pd.DataFrame({'link': processor.new_links, 'date': processor.url_dates})

### 4. Process and filter

In [5]:
processor = library.processLinks(data)
processor.run()
result = processor.df[['link', 'date', 'league', 'league_cleaned', 'teams_cleaned']]
mask = (result.league_cleaned.isin(league_selection)) | (result.teams_cleaned.isin(team_selection))
i = len(result)
result = result[mask].drop(['league_cleaned', 'teams_cleaned'], axis = 1).rename({'league':'league_name'}, axis = 1)
print(f"{i} links before filtering. Committing {len(result)} links.")
result['time_processed'] = datetime.now()
result['retrieved'] = np.nan

902 links before filtering. Committing 182 links.


### 5. Commit

In [6]:
# Establish connection to the SQLite database
conn = sqlite3.connect(library.bronze_path)

result.to_sql('b1_xg_links', conn, if_exists='append', index=False)

conn.close()